# REMIND Electricity Sector Biosphere Flows

Calculate all biosphere flow coefficients for the REMIND electricity sector. **Values are given per kWh**.

In [1]:
from progressbar import progressbar
import helpers.eimod as eimod
import helpers.ei2rmnd as ei2rmnd

Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

In [2]:
%run initialize_notebook.ipynb

In [3]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

## Bioflows without double counting

Calculate impact for an electricity generating technology in the REMIND power sector excluding *other* activities that are part of the electricity sector to avoid double counting.

In [56]:
def rmnd_bioflows(region, scenario="BAU", year=2030, tech_primers={}, double_counting=False):
    """ Extract bioflows for the REMIND electricity sector in a REMIND region.
    
    In this version, for every REMIND technology, we exclude all other technologies
    when performing the LCA to avoid double counting.
        
    A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
    REMIND technology x.
    """
    eidb_name = ei2rmnd.get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)

    result = {}
    bio_names = []
    regions = ei2rmnd.ei_locations_in_remind_region(region)
    print("Found following regions for {}: {}".format(region, regions))
    
    actvts_by_tech = {}
    
    # populate activity dictionary
    print("Collecting ecoinvent activities:")
    for tech, act_list in progressbar(
        eimod.available_electricity_generating_technologies.items(), 
        prefix="Collecting ecoinvent activities: "):
        
        actvts_by_tech[tech] = [act 
                                for act_name in act_list 
                                for act in ei2rmnd.find_activities_by_name(act_name, eidb)]
    
    # flat activity list (needed for excludes)
    all_actvts = [
        act 
        for act_lst in actvts_by_tech.values()
        for act in act_lst] 

    for tech, all_tech_actvts in actvts_by_tech.items():
        print("Processing RMND tech `{}`".format(tech))

        # region specific techs that are adressed
        actvts = [act for act in all_tech_actvts if act["location"] in regions]
        
        if len(actvts) == 0:
            print("No activities found for {}".format(tech))
            continue
        
        demand = 1.
        
        # CHP plants are a problem, not only because they provide energy in megajoule units
        # find out which part is electric energy
        if "CHP" in tech:
            demand = 1./3.6
            
        # normal multi-region and multi-tech lca
        lca = ei2rmnd.multi_lca_average(actvts, demand)
        
        if not double_counting:
            # ... and remove the double-counting
            lca = ei2rmnd.remove_double_counting(lca, actvts, all_actvts)
        
        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        # flows are aggregated by the resp. material
        # we do not care *where* the material flows to
        result[tech] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [5]:
df = rmnd_bioflows("EUR")

N/A% (0 of 20) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']


100% (20 of 20) |########################| Elapsed Time: 0:00:47 Time:  0:00:47


Processing RMND tech `Biomass IGCC CCS`
Bionames changed, old length: 0
new length: 2079
Processing RMND tech `Biomass IGCC`
Processing RMND tech `Coal IGCC`
Processing RMND tech `Coal IGCC CCS`
Processing RMND tech `Coal PC CCS`
Processing RMND tech `Gas CCS`
Processing RMND tech `Biomass CHP`
Processing RMND tech `Coal PC`
Processing RMND tech `Coal CHP`
Processing RMND tech `Gas OC`
Processing RMND tech `Gas CC`
Processing RMND tech `Gas CHP`
Processing RMND tech `Geothermal`
Processing RMND tech `Hydro`
Processing RMND tech `Hydrogen`
No activities found for Hydrogen
Processing RMND tech `Nuclear`
Processing RMND tech `Oil`
Processing RMND tech `Solar CSP`
Processing RMND tech `Solar PV`
Processing RMND tech `Wind`


In [6]:
df = df.reset_index().rename({"level_0": "RMND Tech"}, axis=1).set_index(["RMND Tech", "flow"])
df.sample(20)

,,amount
RMND Tech,flow,
Geothermal,Ametryn,0.000000e+00
Hydro,Benzaldehyde,4.293021e-10
Nuclear,"Methane, tetrachloro-, R-10",1.179178e-10
Gas CHP,"Kieserite, 25% in crude ore, in ground",2.912557e-11
Biomass IGCC,Folpet,0.000000e+00
Solar CSP,Barite,3.319591e-06
Biomass CHP,Monophenyltin,4.776898e-32
Geothermal,Hydrogen sulfide,7.682781e-08
Coal PC,"Energy, kinetic (in wind), converted",3.984163e-08


## Bioflows from the electricity sector with double counting

E.g., flows from wind power plants include coal-generated electricity impacts that are already being accounted for by a `electricity production, coal` lca.

In [7]:
df_dc = rmnd_bioflows("EUR", double_counting=True)

N/A% (0 of 20) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']


100% (20 of 20) |########################| Elapsed Time: 0:00:48 Time:  0:00:48


Processing RMND tech `Biomass IGCC CCS`
Bionames changed, old length: 0
new length: 2079
Processing RMND tech `Biomass IGCC`
Processing RMND tech `Coal IGCC`
Processing RMND tech `Coal IGCC CCS`
Processing RMND tech `Coal PC CCS`
Processing RMND tech `Gas CCS`
Processing RMND tech `Biomass CHP`
Processing RMND tech `Coal PC`
Processing RMND tech `Coal CHP`
Processing RMND tech `Gas OC`
Processing RMND tech `Gas CC`
Processing RMND tech `Gas CHP`
Processing RMND tech `Geothermal`
Processing RMND tech `Hydro`
Processing RMND tech `Hydrogen`
No activities found for Hydrogen
Processing RMND tech `Nuclear`
Processing RMND tech `Oil`
Processing RMND tech `Solar CSP`
Processing RMND tech `Solar PV`
Processing RMND tech `Wind`


In [8]:
df_dc = df_dc.reset_index().rename({"level_0": "RMND Tech"}, axis=1).set_index(["RMND Tech", "flow"])
df_dc.sample(20)

amount
RMND Tech        flow                                                            
Biomass CHP      Bromacil                                            0.000000e+00
Wind             Terpenes                                            2.200663e-10
Gas OC           Transformation, to dump site                        2.265396e-07
Wind             Copper, Cu 6.8E-1%, in mixed ore, in ground         6.712763e-09
Biomass IGCC     Transformation, from pasture, man made, extensive   3.378000e-13
Biomass CHP      TiO2, 54% in ilmenite, 2.6% in crude ore, in gr...  1.563846e-06
                 Flumiclorac-pentyl                                  1.207925e-13
Nuclear          Naphthalene                                         3.080183e-18
Biomass IGCC CCS Iodine-131                                          7.289948e-05
Coal CHP         Fluquinconazole                                     4.504455e-17
Coal IGCC CCS    Transformation, from forest, secondary (non-use)    1.050826e-07
Hydro            COD, Chemical Oxygen Demand                         8.633239e-06
Biomass CHP      Lead                                                1.171329e-07
Hydro            Sulfosate                                           4.145679e-14
Geothermal       Diphenylether-compound                              0.000000e+00
Gas CC           Lactic acid                                         7.530093e-14
Oil              Monocrotophos                                       1.396355e-11
Gas OC           Transformation, to annual crop, irrigated, exte...  0.000000e+00
Gas CCS          Furathiocarb                                        0.000000e+00
Coal PC CCS      Napropamide                                         1.979178e-11

## Impact of double counting

Compare versions with and without double counting.

In [9]:
fulldf = df.join(df_dc, lsuffix="_nodc", rsuffix="_dc")

In [97]:
fulldf["rel. (nodc/dc)"] = fulldf["amount_nodc"]/fulldf["amount_dc"]

In [11]:
fulldf["abs. (dc - nodc)"] = fulldf["amount_dc"] - fulldf["amount_nodc"]

In [98]:
fulldf.loc[fulldf["rel. (nodc/dc)"] > 0].sort_values("rel. (nodc/dc)", ascending=False)[:30]

amount_nodc  \
RMND Tech        flow                                                               
Biomass IGCC     Transformation, to dump site, residual material... -2.933052e-08   
                 Transformation, from dump site, residual materi... -2.933344e-08   
Biomass IGCC CCS Cadmium                                            -2.554539e-09   
Coal IGCC        Transformation, to dump site, sanitary landfill    -2.287068e-09   
                 Transformation, from dump site, sanitary landfill  -2.287068e-09   
Coal IGCC CCS    Transformation, from dump site, inert material ... -2.307214e-08   
                 Transformation, to dump site, inert material la... -2.307214e-08   
Biomass IGCC CCS Lead                                               -1.641552e-07   
Coal IGCC CCS    Cyanide                                            -4.721098e-09   
Biomass IGCC CCS Calcium, ion                                       -1.684903e-03   
Gas CCS          Antimony                                            1.688398e-08   
Biomass IGCC CCS Aluminium                                          -1.351901e-04   
                 Beryllium                                          -1.323643e-07   
                 Manganese                                          -8.709922e-05   
                 Hydrogen sulfide                                   -4.067520e-06   
Coal IGCC CCS    Transformation, from pasture, man made             -3.634913e-06   
Coal IGCC        Cyanide                                            -5.156816e-09   
Biomass IGCC CCS Titanium, ion                                      -4.649831e-06   
Coal PC CCS      Transformation, to dump site, residual material... -2.911734e-08   
                 Transformation, from dump site, residual materi... -2.911847e-08   
Coal IGCC CCS    Tungsten                                           -3.097941e-08   
Coal IGCC        Transformation, from pasture, man made             -3.009112e-06   
Biomass IGCC     Carbon                                             -7.738301e-06   
                 Beryllium                                          -3.567833e-07   
Coal PC CCS      Iodide                                             -6.439882e-09   
Coal IGCC        Tungsten                                           -2.985480e-08   
Gas OC           Antimony                                            2.315695e-08   
Nuclear          Antimony                                            4.393911e-08   
Gas CC           Antimony                                            1.432330e-08   
Biomass IGCC     Oxygen                                             -7.641486e-04   

                                                                        amount_dc  \
RMND Tech        flow                                                               
Biomass IGCC     Transformation, to dump site, residual material... -4.063964e-09   
                 Transformation, from dump site, residual materi... -4.171702e-09   
Biomass IGCC CCS Cadmium                                            -4.145618e-10   
Coal IGCC        Transformation, to dump site, sanitary landfill    -7.795160e-10   
                 Transformation, from dump site, sanitary landfill  -7.795160e-10   
Coal IGCC CCS    Transformation, from dump site, inert material ... -8.974355e-09   
                 Transformation, to dump site, inert material la... -8.974355e-09   
Biomass IGCC CCS Lead                                               -6.809059e-08   
Coal IGCC CCS    Cyanide                                            -2.462278e-09   
Biomass IGCC CCS Calcium, ion                                       -9.414095e-04   
Gas CCS          Antimony                                            9.905316e-09   
Biomass IGCC CCS Aluminium                                          -8.456409e-05   
                 Beryllium                                          -8.609710e-08   
                 Manganese                                          -5.763

In [ ]:
fulldf.loc[fulldf["rel. (nodc/dc)"] > 1].sort_values(by="rel. (nodc/dc)", ascending=False)

### What about geothermal power plants?

...seem to have quite a large double-counting value (large electricity production?).

In [14]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

In [15]:
eidb = Database("ecoinvent_Remind_BAU_2030")

In [16]:
regions = ei2rmnd.ei_locations_in_remind_region("EUR")
geot = [act for act in eidb if "electricity production" in act["name"]
and "geothermal" in act["name"]
       and act["location"] in regions]

In [17]:
geot

['electricity production, deep geothermal' (kilowatt hour, CZ, None),
 'electricity production, deep geothermal' (kilowatt hour, GB, None),
 'electricity production, deep geothermal' (kilowatt hour, LT, None),
 'electricity production, deep geothermal' (kilowatt hour, FR, None),
 'electricity production, deep geothermal' (kilowatt hour, HU, None),
 'electricity production, deep geothermal' (kilowatt hour, AT, None),
 'electricity production, deep geothermal' (kilowatt hour, PL, None),
 'electricity production, deep geothermal' (kilowatt hour, DE, None),
 'electricity production, deep geothermal' (kilowatt hour, PT, None),
 'electricity production, deep geothermal' (kilowatt hour, IT, None),
 'electricity production, deep geothermal' (kilowatt hour, LV, None)]

In [18]:
lca = LCA({act: 1./len(geot) for act in geot})

In [19]:
lca.lci()

In [20]:
bio = Database("biosphere3")

In [21]:
ant=[act for act in bio if "Antimony" == act["name"]]

In [87]:
ant_idcs = []
for act in ant:
    try:
        ant_idcs.append(lca.biosphere_dict[act.key])
    except KeyError as e:
        print("Flow not found: {}".format(get_activity(act.key)["name"]))

Flow not found: Antimony
Flow not found: Antimony
Flow not found: Antimony


In [88]:
ant_contr = lca.inventory[ant_idcs].sum(axis=0).tolist()[0]

In [90]:
names = [get_activity(key)["name"] for key in lca.activity_dict]

In [94]:
contrbtrs = pd.Series({"{} | {}".format(names[idx], list(lca.activity_dict.keys())[idx]): ant_contr[idx] for idx in range(len(names))})

In [95]:
contrbtrs.sort_values()[:20]

treatment of average incineration residue, residual material landfill | ('ecoinvent_Remind_BAU_2030', '0865a0269c470058297ce25173fbb792')   -7.018001e-08
treatment of municipal solid waste, incineration | ('ecoinvent_Remind_BAU_2030', '215490c7d59cde0315fa5140599c7faa')                        -2.832708e-10
treatment of waste plastic, mixture, municipal incineration | ('ecoinvent_Remind_BAU_2030', '6f23285fb660e7ab995cccfd0d7d5d7b')             -2.387445e-11
treatment of waste polyvinylchloride, municipal incineration | ('ecoinvent_Remind_BAU_2030', 'c92f62f9bfe4700e55be537c361407e5')            -4.641754e-13
treatment of waste polystyrene, municipal incineration | ('ecoinvent_Remind_BAU_2030', 'e99e8a88dcf55998b1a36347d30cf305')                  -2.791835e-13
treatment of waste polyethylene, municipal incineration | ('ecoinvent_Remind_BAU_2030', '13540994cba6321baba8206065613d05')                 -1.949326e-14
treatment of waste polypropylene, municipal incineration | ('ecoinvent_Remin

In [96]:
contrbtrs.sum()

-5.989757472444638e-09

The negative impact seems to come from bioenergy power plants.